In [1]:
!pip install tensorflow mediapipe

import os
import numpy as np
import pickle
from tqdm import tqdm
import tensorflow as tf
import mediapipe as mp
import cv2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 47.3 MB/s eta 0:00:00


In [3]:
import os
import pickle
from tqdm import tqdm
import cv2
import tensorflow as tf
import mediapipe as mp
from concurrent.futures import ProcessPoolExecutor

# GPU setup with minimal memory growth and precision
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        try:
            tf.config.experimental.set_memory_growth(device, True)
            tf.config.set_logical_device_configuration(
                device, [tf.config.LogicalDeviceConfiguration(memory_limit=10000)]
            )
            print("GPU configured with a memory limit of 10000 MB.")
        except Exception as e:
            print(f"Error configuring GPU: {e}")
else:
    print("No GPU detected, running on CPU.")

try:
    tf.keras.mixed_precision.set_global_policy('mixed_float16')
    print("Mixed precision enabled for speedup.")
except ValueError:
    print("Mixed precision not supported, running with default precision.")

# Function to save progress
def save_progress(filepath, data):
    with open(filepath, 'wb') as f:
        pickle.dump(data, f)

# Function to load progress
def load_progress(filepath):
    if os.path.exists(filepath):
        try:
            with open(filepath, 'rb') as f:
                return pickle.load(f)
        except Exception as e:
            print(f"Error loading checkpoint {filepath}: {e}")
    return []

# Function to extract landmarks from a single image
def extract_landmarks_from_image(image_path):
    """Extract facial landmarks using MediaPipe from a given image."""
    mp_face_mesh = mp.solutions.face_mesh
    landmarks = [(0, 0, 0)] * 468  # Default empty landmarks in case of failure

    try:
        image = cv2.imread(image_path)
        if image is None:
            print(f"Failed to read image: {image_path}")
            return image_path, landmarks

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True) as face_mesh:
            results = face_mesh.process(image_rgb)
            if results.multi_face_landmarks:
                face_landmarks = results.multi_face_landmarks[0]
                landmarks = [(lm.x, lm.y, lm.z) for lm in face_landmarks.landmark]
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")

    return image_path, landmarks

# Wrapper function for multiprocessing
def process_image(image_path):
    return extract_landmarks_from_image(image_path)

# Function to process images and save landmarks with checkpoints using multiprocessing
def extract_landmark_features_with_checkpoint(image_paths, checkpoint_path):
    saved_progress = load_progress(checkpoint_path)
    processed_images = {entry['image_path'] for entry in saved_progress}
    remaining_images = [img for img in image_paths if img not in processed_images]

    print(f"Total images: {len(image_paths)}, Remaining images: {len(remaining_images)}")

    chunk_size = max(1, len(remaining_images) // (8 * 4))  # Optimize chunk size for batching
    with ProcessPoolExecutor(max_workers=8) as executor:  # Use process-based parallelism
        for i in tqdm(range(0, len(remaining_images), chunk_size), desc="Extracting Landmark Features"):
            chunk = remaining_images[i:i + chunk_size]
            results = list(executor.map(process_image, chunk))

            # Append results and save progress
            saved_progress.extend({'image_path': img_path, 'landmarks': lm} for img_path, lm in results)
            save_progress(checkpoint_path, saved_progress)  # Save after processing each chunk

    return saved_progress

if __name__ == "__main__":
    # Define paths and directories
    base_path = 'drive/MyDrive/validation/real_valid/'
    os.makedirs(base_path, exist_ok=True)

    real_images_path = 'drive/MyDrive/validation/real_valid/real/'

    real_checkpoint = os.path.join(base_path, 'landmarks_real_valid.pkl')

    # Collect image paths efficiently
    def get_image_paths(folder_path):
        return [
            os.path.join(root, file)
            for root, _, files in os.walk(folder_path)
            for file in files if file.lower().endswith(('.jpg', '.jpeg', '.png'))
        ]

    real_image_paths = get_image_paths(real_images_path)

    # Extract landmark features
    print("Processing real images...")
    landmark_features_real = extract_landmark_features_with_checkpoint(real_image_paths, real_checkpoint)

    print("Landmark feature extraction completed.")


Error configuring GPU: Cannot set memory growth on device when virtual devices configured
Mixed precision enabled for speedup.
Processing real images...
Total images: 1548, Remaining images: 1548


Extracting Landmark Features: 100%|██████████| 33/33 [01:34<00:00,  2.88s/it]

Landmark feature extraction completed.


In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
import pickle
from tqdm import tqdm
import cv2
from concurrent.futures import ThreadPoolExecutor

# GPU setup
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        try:
            tf.config.experimental.set_memory_growth(device, True)
            tf.config.set_logical_device_configuration(
                device, [tf.config.LogicalDeviceConfiguration(memory_limit=10000)]  # Adjust as needed
            )
            print(f"Configured GPU with a memory limit of 10000 MB.")
        except Exception as e:
            print(f"Error configuring GPU: {e}")
else:
    print("No GPU detected, running on CPU.")

# Mixed precision setup
try:
    tf.keras.mixed_precision.set_global_policy('mixed_float16')
    print("Mixed precision enabled for speedup.")
except ValueError:
    print("Mixed precision not supported, running with default precision.")

# Save and load progress functions
def save_progress(filepath, data):
    try:
        with open(filepath, 'wb') as f:
            pickle.dump(data, f)
        print(f"Progress saved at: {filepath}")
    except Exception as e:
        print(f"Error saving progress: {e}")

def load_progress(filepath):
    if os.path.exists(filepath):
        try:
            with open(filepath, 'rb') as f:
                return pickle.load(f)
        except Exception as e:
            print(f"Error loading progress: {e}")
    return []

# Image preprocessing
def preprocess_image(image_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            print(f"Failed to read image: {image_path}")
            return np.zeros((224, 224, 3), dtype=np.float32)

        image = cv2.resize(image, (224, 224))
        image = preprocess_input(image.astype(np.float32))
        return image
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return np.zeros((224, 224, 3), dtype=np.float32)

# Multithreaded preprocessing
def preprocess_images_multithreaded(image_files, num_threads=4):
    processed_images = []
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        results = list(tqdm(executor.map(preprocess_image, image_files), total=len(image_files), desc="Preprocessing images"))
        processed_images.extend(results)
    return np.array(processed_images, dtype=np.float32)

# Spatial feature extraction for individual images
def extract_spatial_features_for_images(image_dir, model, checkpoint_path, batch_size=32, save_interval=200, num_threads=4):
    features = load_progress(checkpoint_path) or []
    processed_images = {f['image_name'] for f in features if 'image_name' in f}
    all_images = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    remaining_images = [img for img in all_images if img not in processed_images]

    print(f"Total images: {len(all_images)}, Remaining images: {len(remaining_images)}")

    for i in tqdm(range(0, len(remaining_images), batch_size), desc="Extracting spatial features"):
        batch_files = remaining_images[i:i+batch_size]

        try:
            # Multithreaded preprocessing
            processed_batch = preprocess_images_multithreaded(batch_files, num_threads)
            dataset = tf.data.Dataset.from_tensor_slices(processed_batch).batch(batch_size).prefetch(tf.data.AUTOTUNE)

            features_batch = model.predict(dataset, verbose=1)

            for img_path, feature in zip(batch_files, features_batch):
                features.append({'image_name': img_path, 'features': feature})
        except Exception as e:
            print(f"Error extracting features for batch {i // batch_size + 1}: {e}")

        # Save progress periodically
        if len(features) % save_interval == 0:
            save_progress(checkpoint_path, features)

    save_progress(checkpoint_path, features)
    return features

if __name__ == "__main__":
    # Define paths
    base_path = 'drive/MyDrive/validation/fake_valid/'
    features_dir = 'drive/MyDrive/validation/fake_valid/'
    os.makedirs(features_dir, exist_ok=True)

    real_images_dir = os.path.join(base_path, 'fake')
    real_checkpoint = os.path.join(features_dir, 'spatial_valid_fake.pkl')

    # Load EfficientNet model
    efficientnet = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')

    # Extract spatial features for real images
    print("Processing real images...")
    spatial_features_real = extract_spatial_features_for_images(
        image_dir=real_images_dir,
        model=efficientnet,
        checkpoint_path=real_checkpoint,
        batch_size=32,
        save_interval=200,
        num_threads=4
    )

    print("Spatial feature extraction for images completed.")

Error configuring GPU: Cannot set memory growth on device when virtual devices configured
Mixed precision enabled for speedup.
Processing real images...
Total images: 1524, Remaining images: 1524


Preprocessing images: 100%|██████████| 32/32 [00:14<00:00,  2.18it/s]


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


Preprocessing images: 100%|██████████| 32/32 [00:00<00:00, 278.69it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


Preprocessing images: 100%|██████████| 20/20 [00:00<00:00, 177.54it/s]


1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step


Extracting spatial features: 100%|██████████| 48/48 [00:41<00:00,  1.15it/s]

Progress saved at: drive/MyDrive/validation/fake_valid/spatial_valid_fake.pkl
Spatial feature extraction for images completed.
